<h1>Welcome to our program!<h/3>
<h3>This interactive tool displays choropleth maps to aid in visualization of HIV statistics. This program is intended to identify areas within the USA which need further attention in the areas of sexual health and HIV prevention.<h/3>
<br>   
<small> Created by Tyler Audino, Tikiri Ekanayake, Arlen Gyden, and Quinn Yuan

In [ ]:
#Here are the packages we used
import pandas as pd
import json #for geojson files
import plotly.express as px
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

## Data Cleaning - DO NOT RUN

In [ ]:
#Arlen - I won't run this code because the excel file has already been made 
#citation: https://www.geeksforgeeks.org/how-to-merge-multiple-excel-files-into-a-single-files-with-python/

#specifying the path to excel files
path = "/Users/larry-gyden/Documents/Capstone_Raw_Data/"
 
#excel files in the path
file_list = [path+"AIDSVu_State_NewDX_2008-1.xlsx", path+"AIDSVu_State_NewDX_2009-1.xlsx",
            path+"AIDSVu_State_NewDX_2010-1.xlsx", path+"AIDSVu_State_NewDX_2011-1.xlsx",
            path+"AIDSVu_State_NewDX_2012.xlsx", path+"AIDSVu_State_NewDX_2013.xlsx",
            path+"AIDSVu_State_NewDX_2014.xlsx", path+"AIDSVu_State_NewDX_2015.xlsx",
            path+"AIDSVu_State_NewDX_2016.xlsx", path+"AIDSVu_State_NewDX_2017-1.xlsx",
            path+"AIDSVu_State_NewDX_2018.xlsx", path+"AIDSVu_State_NewDX_2019.xlsx",
            path+"AIDSVu_State_NewDX_2020.xlsx"]
 
excl_list = []

#iterates through the file list appending each file to one excel sheet
for file in file_list:
    excl_list.append(pd.read_excel(file))

#concatenate all excel files into a single excel sheet
excl_merged = pd.concat(excl_list, ignore_index=True)
 
#exports the excel sheet
excl_merged.to_excel(path+'Capstone_Raw_Data_Merged.xlsx', index=False)

## Map Generator

In [ ]:
#loading JSON file for map outline 
#JSON file source: https://www.kaggle.com/datasets/pompelmo/usa-states-geojson?resource=download 
usamaps = json.load(open("us-states.json", 'r'))

In [ ]:
#creating dataframe of excel sheet
data = pd.read_excel("capstonedata.xlsx")

In [ ]:
#displays headers, and confirms that the data was imported sucessfully
data.head()

In [ ]:
#Tyler
def branchmenu():
    
    "Returns the name of the column from the dataframe based on the user's inputs for parameters"
    
    print("We have data in regards to gender, race, age, and transmission.")
    print("You can select up to two parameters at a time.") #only two parameters because available data is limited to this
    
    parameters = selection() #parameters is a list of parameters determined by the selection function
    narrowedlist = []  #for narrowing down to the column headers that align with the user input
    narrowedlist2 = []
    headerlist = list(data.columns.values) #list of header values for use with for loop
    headerlist2 = [] #list for getting rid of stability when 'rate' is chose
    datatype = datatypemenu() #menu to ask person what data type they want to look at 
    
    if datatype == 'Rate':
        for header in headerlist:
            if 'Stability' not in header:
                headerlist2.append(header)
    else: 
        headerlist2 = headerlist
            
    if len(parameters) != 0:
        for header in headerlist2: #identify columns that fit these first parameter
            if parameters[0] in header: #iterate through the column names to find if
                narrowedlist.append(header)      
    
    if len(parameters) == 2:
        for header in narrowedlist:
            if parameters[1] in header:
                narrowedlist2.append(header)
    
    
    finalselection = []    
        
    if len(narrowedlist2) != 0:  #narrowing down when 2 parameters chosen
        for header in narrowedlist2:
            if datatype in header:
                finalselection.append(header)
    else:   #narrowing down when 1 parameter chosen
        newlist = []
        for header in narrowedlist:
            if 'and' not in header:
                newlist.append(header)    
        for header in newlist:   
            if datatype in header:
                finalselection.append(header)
                
    if len(parameters) == 0: #when no parameters chosen (aka "ALL")            
        finalselection.clear() 
        if datatype == "Cases":
            finalselection.append("New Diagnoses State Cases")
        elif datatype == "Rate":
            finalselection.append("New Diagnoses State Rate")
    
    if len(finalselection) > 1:
        finalselection = uhohparameters(finalselection)
    else:     
        finalselection = str(finalselection[0])
        
    print(finalselection)    
    
    return finalselection
    



In [ ]:
#Tyler
def selection():
    selectionlist = []
    
    #indicates how many parameters are chosen at a given moment
    selectioncounter = 0 
    
    #gender
    selectsex = input("Would you like to look at a specific sex? (Type Y or N): ")

    while selectioncounter != 2: 
        if selectsex == 'Y' or selectsex == 'y':
            #set equal to gender variable
            sex = sexmenu()
            selectioncounter += 1
            selectionlist.append(sex)
            
        #race
        selectrace = input("Would you like to look at a specific race? (Type Y or N): ")

        if selectrace == 'Y' or selectrace == 'y':
            #set equal to race variable
            race = racemenu() 
            selectioncounter += 1
            selectionlist.append(race)
        
        if selectioncounter == 2:
            break
            
        #age
        selectage = input("Would you like to look at a specific age range? (Type Y or N): ")

        if selectage == 'Y' or selectage == 'y':
            age = agemenu() #set equal to age variable
            selectioncounter += 1
            selectionlist.append(age)
       
        if selectioncounter == 2:
            break 
            
        #transmission
        selecttrans = input("Would you like to look at a specific transmission type? (Type Y or N): ")

        if selecttrans == 'Y' or selectage == 'y':
            transmission = transmissionmenu()
            selectioncounter += 1
            selectionlist.append(transmission)
            
        break
            
    return selectionlist
        

In [ ]:
#Arlen
def racemenu():
    "Identifies user's parameter selection for race"
    raceselection = str(input("Select from the following races: Black, White, Hispanic, Asian, American Indian/Alaska Native, Native Hawaiian/Other Pacific Islander, Multiracial: "))
    
    while raceselection != "Black" and raceselection != "White" and raceselection != "Hispanic" and raceselection != "Asian" and raceselection != "American Indian/Alaska Native" and raceselection != "Native Hawaiian/Other Pacific Islander" and raceselection != "Multiracial":
        raceselection = input("Oops! You did not correctly type your selection. ")
    
    return raceselection

In [ ]:
#Arlen
def agemenu():
    "Identifies user's parameter selection for age"
    ageselection = str(input("Select from the following age groups: 13-24, 25-34, 35-44, 45-54, 55+ "))
    while ageselection != "13-24" and ageselection != "25-34" and ageselection != "35-44" and ageselection != "45-54" and ageselection != "55+":
        ageselection = input("Oops! You did not correctly type your selection. ")
    return ageselection

In [ ]:
#Arlen
def sexmenu():
    "Identifies user's parameter selection for sex"
    sexselection = str(input("Select which sex: Male, Female "))
    while sexselection != "Male" and sexselection != "Female":
        sexselection = input("Oops! You did not correctly type your selection. ")
    return sexselection

In [ ]:
#Tyler 
def transmissionmenu():
    
    "Identifies user's parameter selection for transmission type"
    
    tselection = str(input("Select which transmission type: Heterosexual Contact, IDU, MSM, Other "))
    while tselection != "Heterosexual Contact" and tselection != "IDU" and tselection != "MSM" and tselection != "Other":
        tselection = input("Oops! You did not correctly type your selection. ")
    return tselection

In [ ]:
#Arlen
def datatypemenu():
    typeselection = input("Select from the following forms of data: Cases, Rate ")
    while typeselection != "Cases" and typeselection != "Rate":
        typeselection = input("Oops! You did not correctly type your selection. ")
    return typeselection

In [ ]:
#Tyler
def uhohparameters(oldheaders):
    
    "For use when the program narrows down the header names to more than one header"
    
    print("Oops! Our program found two possible datasets.")
    print("Your options are: " + str(oldheaders))
    newheader = input("Please type the name of the dataset EXACTLY as displayed (no quotation marks): ")
    while newheader != oldheaders[0] and newheader != oldheaders[1]:
        newheader = input("Oops! You did not correctly type your selection. ")
        
    return newheader

In [ ]:
#Arlen
class maps():
    
    "Generates a map with specified dataset"
    
    def __init__(figure, dataframe, locations, geojson, color, scope):
        figure.dataframe = dataframe
        figure.color = color
        figure.locations = locations
        figure.geojson = geojson
        figure.scope = scope
    
    def generatemap(figure):
        px.choropleth(figure.dataframe, color = figure.color, locations = figure.locations, 
                      scope = figure.scope, geojson = figure.geojson).show("notebook")
        
    def animatemap(figure):
        px.choropleth(figure.dataframe, color = figure.color, locations = figure.locations, 
                      scope = figure.scope, geojson = figure.geojson, 
                      animation_frame = 'Year').show("notebook")

In [ ]:
#Arlen
def dataskip(colordata, states = False):
    
    "Iterates through user's selection column and skips unusable data points"
    
    #converts raw data into numpy array
    rawdata = data[["Year", "State Abbreviation", colordata]].to_numpy()
    usabledata = []
    
    #iterates through array and skips any unusable data types
    for datapoint in range(len(rawdata)):
        if type(rawdata[datapoint, 2]) == str or rawdata[datapoint, 2] <= 0:
            continue
        usabledata.append(rawdata[datapoint, :])
    
    #converts numpy array back to dataframe of the usable data
    return pd.DataFrame(usabledata, columns = ["Year", "State Abbreviation", colordata])

In [ ]:
#Arlen
def mapselect(generate = True, animate = True):
    "User's function that allows them to select parameters and generate desired map"
    data = dataskip(branchmenu())
    columnnames = list(data.columns)
    
    #specify user inputs to maps class from branch menu and dataskip functions
    mapchoice = maps(data, locations = columnnames[1], geojson = usamaps,  
                     color = columnnames[2], scope = "usa")
    
    if generate == True and animate == False:
        return mapchoice.generatemap()
    
    if generate == True and animate == True:
        return mapchoice.animatemap()

In [ ]:
mapselect()

## ANOVA - Tikiri

In [ ]:
# reading the data frame in
dataset = pd.read_excel('anova_data_frame.xlsx')

In [ ]:
menu = '1. gender_age\n' \
       '2. gender_race\n' \
       '3. gender_transmission\n' \
       '4. transmission_age\n' \
       '5. transmission_race\n' \
       '6. age_race\n'

print(menu)



# asking for choice input + making sure it is a valid answer
choice = input('Select which categories to compare: ')

while (choice != 'gender_age') and (choice != 'gender_race') and (choice != 'gender_transmission') and (choice != 'transmission_age') and (choice != 'transmission_race') and (choice != 'age_race'):
    print('Invalid choice.')
    choice = input('Select which categories to compare: ')


    
# asking for year input + making sure it is a valid answer
print('Select a year to look at from 2008-2020. Type 0 to use all the data.')
year = int(input('Year: '))

while (year != 0) and (2020 < year) and (year < 2008):
    print('Invalid year.')
    year = int(input('Year: '))

    

# seeing if I need to specify a certain year w/in the dataset, or use the whole thing
if year != 0:
    observed_year = dataset[(dataset['Year'] == year)]
elif year == 0 :
    observed_year = dataset
        


# which cases w/in the dataset to look at it
dependent = choice + '_cases'
    # ex: gender_age_cases

    

# specifying name of the first column for the category
categories = choice.split('_')
first_iv = [categories[0], categories[0], categories[1]]
first_iv = '_'.join(first_iv)
    # ex: gender_gender_age


    
# second column name
second_iv = [categories[1], categories[0], categories[1]]
second_iv = '_'.join(second_iv)
    # ex: age_gender_age
    


# the string that goes w/in the first ols argument
the_stuffing = f'{dependent} ~ C({first_iv}) + C({second_iv}) +\
C({first_iv}):C({second_iv})'



# code for the actual anova table
model = ols(the_stuffing,
           data = observed_year).fit()
result = sm.stats.anova_lm(model, type = 2)



print(result)
print('\n\n')
print('If the PR(>F) value [aka the P-value] is less than 0.05, then that factor has a statistically significant effect' \
      ' on the data.')
print('The C(factor):C(factor) row provides information on the interaction between the two factors.')

## T-Test - Quinn

In [ ]:
#just displaying the merged data excel sheet
pd.read_excel('capstonedata.xlsx')

In [ ]:
#reading the excel sheet into a pandas DataFrame
df = pd.read_excel('capstonedata.xlsx')

In [ ]:
#T-test between variables 
choice = input('Select a state to compare new Diagnoses male and female cases: ')

#cleaning the data by the state assigned according to user's input state
state = df[(df['State'] == choice)]

#The actual t-test calculations
sample1 = state['New Diagnoses Male Cases']
sample2 = state['New Diagnoses Female Cases']
ttest_result = ttest_ind(sample1, sample2)

print(ttest_result)

#extracting the p-value from the scipy result 
pvalue = ttest_result.pvalue

#displaying an analysis according to the t-value and the p-value
if(pvalue<0.05):
    #reject the null hypothesis - there is statistically 
    #significant difference between the samples
    print("Since the P-value is smaller than the alpha value (0.05), there is statistically significant difference " \
          "between the  samples you chosen")
elif (pvalue>0.05):
    #fall to reject the null hypothesis - there is NO statistically 
    #significant difference between the samples 
    print("Since the T-value is larger than the alpha value (0.05), there is NO statistically significant " \
          "difference between the samples you chosen")
else: 
    print("Your selecting data is invalid for T-test calculation")
    

In [ ]:
#displaying box plot graph

#cleaning the data for the box plot
da = state[["New Diagnoses Male Cases", "New Diagnoses Female Cases"]]

#graphing
plt.figure(figsize=(7, 7, ))
graph = plt.boxplot(da)
plt.xticks([1, 2], ["New Diagnoses Male Cases", "New Diagnoses Female Cases"])

plt.show()